## Predicting Kobe's shots using XGBOOST##

This notebook built on the analysis in: https://www.kaggle.com/selfishgene/kobe-bryant-shot-selection/psychology-of-a-professional-athlete

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb

### Reading data

In [2]:
data = pd.read_csv('../input/data.csv')
data.set_index('shot_id', inplace=True)

The following should be explicitly made categorical:

Let's prepare the data for the splitting into train/test set later.

In [3]:
unknown_mask = data['shot_made_flag'].isnull()
data_cl = data.copy()
target = data_cl['shot_made_flag'].copy()

The following columns are to be dropped:

In [4]:
data_cl.drop('team_id', inplace=True, axis=1) #only 1 category
data_cl.drop('lat', inplace=True, axis=1) # correlated with loc_x
data_cl.drop('lon', inplace=True, axis=1) # correlated with loc_y
data_cl.drop('game_id', inplace=True, axis=1) # should not be dependent on game id, furthermore it's contained in opponent/match
data_cl.drop('game_event_id', inplace=True, axis=1) # independent, unique for every shots in a game
data_cl.drop('team_name', inplace=True, axis=1) # always LA Lakers
data_cl.drop('shot_made_flag', inplace=True, axis=1) # target variables

### Feature Engineering ##

As Kobe's psychology is impacted by the number of seconds from the game, let's create more features for these.

In [5]:
# time into the game
data_cl['seconds_from_period_end'] = 60 * data_cl['minutes_remaining'] + data_cl['seconds_remaining']
data_cl['last_5_sec_in_period'] = data_cl['seconds_from_period_end'] < 5
data_cl['seconds_from_period_start'] = 60*(11-data_cl['minutes_remaining'])+(60-data_cl['seconds_remaining'])
data_cl['seconds_from_game_start'] = (data_cl['period'] <= 4).astype(int)*(data_cl['period']-1)*12*60 + (data_cl['period'] > 4).astype(int)*((data_cl['period']-4)*5*60 + 3*12*60) + data_cl['seconds_from_period_start']

# drop redundant features
data_cl.drop('minutes_remaining', axis=1, inplace=True)
data_cl.drop('seconds_remaining', axis=1, inplace=True)

Matchup -- away/home:

In [6]:
data_cl['home_play'] = data_cl['matchup'].str.contains('vs').astype('int')
data_cl.drop('matchup', axis=1, inplace=True)

Extract year and month from date of game:

In [7]:
data_cl['game_date'] = pd.to_datetime(data_cl['game_date'])

# year and month
data_cl['game_year'] = data_cl['game_date'].dt.year
data_cl['game_month'] = data_cl['game_date'].dt.month

# day of week/year
data_cl['dayOfWeek'] = data_cl['game_date'].dt.dayofweek
data_cl['dayOfYear'] = data_cl['game_date'].dt.dayofyear

data_cl.drop('game_date', axis=1, inplace=True)

Replace 20 least common action types with value 'Other'

In [8]:
rare_action_types = data_cl['action_type'].value_counts().sort_values().index.values[:20]
data_cl.loc[data_cl['action_type'].isin(rare_action_types), 'action_type'] = 'Other'

One-hot encoding of categorical variables:

In [9]:
categorial_cols = [
    'action_type', 'combined_shot_type', 'period', 'season', 'shot_type',
    'shot_zone_area', 'shot_zone_basic', 'shot_zone_range', 'game_year',
    'game_month', 'opponent']

for cc in categorial_cols:
    dummies = pd.get_dummies(data_cl[cc])
    dummies = dummies.add_prefix("{}_".format(cc))
    data_cl.drop(cc, axis=1, inplace=True)
    data_cl = data_cl.join(dummies)

### Train/test/validation Splitting ##

In [10]:
data_submit = data_cl[unknown_mask]
# Separate dataset for training
X = data_cl[~unknown_mask]
Y = target[~unknown_mask]

### XGBOOST

In [11]:
d_train = xgb.DMatrix(X, label=Y)
dtest = xgb.DMatrix(data_submit)

In [23]:
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['max_depth'] = 7
params['silent'] = 1
params['colsample_bytree'] = 0.7
params['eta'] = 0.004
params['max_delta_step'] = 1
params['min_child_weight'] = 3

In [24]:
#cvp = xgb.cv(params, d_train, num_boost_round=100000, early_stopping_rounds=10, metrics=['logloss'], verbose_eval=1, stratified=True)
#print("Best is {0} with logloss {1}".format(np.argmin(cvp['test-logloss-mean'] + cvp['test-logloss-std']), np.min(cvp['test-logloss-mean'] + cvp['test-logloss-std']))) 

[0]	train-logloss:0.692389+1.43372e-05	test-logloss:0.692472+8.95669e-06
[1]	train-logloss:0.691713+3.99694e-05	test-logloss:0.691853+8.01679e-05
[2]	train-logloss:0.691116+1.06562e-05	test-logloss:0.691307+6.95126e-05
[3]	train-logloss:0.690468+9.50836e-05	test-logloss:0.690725+3.2968e-05
[4]	train-logloss:0.689813+4.67832e-05	test-logloss:0.690131+0.0001059
[5]	train-logloss:0.689194+8.4727e-05	test-logloss:0.689577+0.000128034
[6]	train-logloss:0.688461+9.90634e-05	test-logloss:0.688925+0.000138089
[7]	train-logloss:0.687754+9.38509e-05	test-logloss:0.688294+0.00016004
[8]	train-logloss:0.687049+0.000133032	test-logloss:0.687659+0.000166664
[9]	train-logloss:0.686389+0.000179752	test-logloss:0.687057+0.000198247
[10]	train-logloss:0.68569+0.000213901	test-logloss:0.686427+0.000213187
[11]	train-logloss:0.685083+0.000279042	test-logloss:0.685876+0.000121219
[12]	train-logloss:0.684481+0.000168644	test-logloss:0.685338+0.00021663
[13]	train-logloss:0.683819+0.000166591	test-logloss:0.

In [25]:
#print("Best is {0} with logloss {1}".format(np.argmin(cvp['test-logloss-mean'] + cvp['test-logloss-std']), np.min(cvp['test-logloss-mean'] + cvp['test-logloss-std']))) 

Best is 961 with logloss 0.6069344787752194


In [15]:
clf = xgb.train(params, d_train, num_boost_round=512)

In [16]:
preds = clf.predict(dtest)
submission = pd.DataFrame()
submission["shot_id"] = data_submit.index
submission["shot_made_flag"]= preds

submission.to_csv("sub_xgb.csv",index=False)